In [1]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib
import pandas as pd

In [2]:
def load_data():
    data = pd.read_csv('anies_prepro.csv', usecols=['text'],dtype=str)
    return data

# Kita akan menggunakan kolom teks saja di tahap ini, tanpa kolom label
tweets = load_data()
tweets

,text
0,RT @eddy_widodo_165: Alhamdulilah 🤲\nBacapres ...
1,RT @renandabachtar: Din Syamsuddin Lebih Senan...
2,RT @PutraErlangga95: Ingin menyelamatkan Indon...
3,"RT @idtodayco: Kurang Bukti, Relawan Ganjar Pr..."
4,RT @Yurissa_Samosir: Sambutan hangat dari Masy...
...,...
2560,RT @rahmaniarbaftim: Untuk bisa menjadi Pemimp...
2561,RT @YepryR: Terimakasih masyarakat brebes jawa...
2562,Anies Baswedan tegaskan tidak akan lanjutkan P...
2563,RT @MissKoral: Petani Brebes Jateng Deklarasi ...


In [3]:
# Memanfaatkan nltk VADER untuk menggunakan leksikon kustom
nltk.download('vader_lexicon')
sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()

# membersihkan leksikon VADER default
sia1A.lexicon.clear()
sia1B.lexicon.clear()
sia2.lexicon.clear()

# Membaca leksikon InSet
# Leksikon InSet lexicon dibagi menjadi dua, yakni polaritas negatif dan polaritas positif;
# kita akan menggunakan nilai compound saja untuk memberi label pada suatu kalimat
with open('_json_inset-neg.txt') as f:
    data1A = f.read()
with open('_json_inset-pos.txt') as f:
    data1B = f.read()

# Membaca leksikon sentiwords_id
with open('_json_sentiwords_id.txt') as f:
    data2 = f.read()

# Mengubah leksikon sebagai dictionary
insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)
senti = json.loads(data2)

# Update leksikon VADER yang sudah 'dimodifikasi'
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)
sia2.lexicon.update(senti)

print(reprlib.repr(sia1A.lexicon))
print(reprlib.repr(sia1B.lexicon))
print(reprlib.repr(sia2.lexicon))

{'(barang) bekas': -4, '(olahraga) bokser': -5, '(tua) uzur': -3, 'Anda': -4, ...}
{'(hujan) gerimis': 1, '(warna) dadu': 3, 'Ahad': 3, 'Sri paduka': 4, ...}
{'abadi': 5, 'absen': -3, 'abu-abu': -1, 'acuh': 4, ...}


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wols88\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
sample = "kalau kamu sudah sampai sini kamu hebat ayo terus kamu pasti bisa"
print("insetNeg: ", sia1A.polarity_scores(sample))
print("insetPos: ", sia1B.polarity_scores(sample))
print("insetCpdSum: 'compound':", sia1A.polarity_scores(sample)["compound"] + sia1B.polarity_scores(sample)["compound"])

print("senti\t: ", sia2.polarity_scores(sample))

insetNeg:  {'neg': 0.526, 'neu': 0.474, 'pos': 0.0, 'compound': -0.875}
insetPos:  {'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.9517}
insetCpdSum: 'compound': 0.07669999999999999
senti	:  {'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.6124}


In [5]:
def is_positive_inset(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

def is_positive_senti(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia2.polarity_scores(tweet)["compound"] > 0

In [6]:
tweets = tweets["text"]
# Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari insetNeg dan insetPos
output = "./anies_inset.csv"
with open(output, 'w') as f:
#     f.write('label\n')
    for tweet in tweets:
        if is_positive_inset(tweet) == True:
            label = "positif"
        else:
            label = "negatif"
#         print(label)
        f.write(str(label+'\n'))

# Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari sentiwords_id
output = "./anies_senti.csv"
with open(output, 'w') as f:
#     f.write('label\n')
    for tweet in tweets:
        if is_positive_senti(tweet) == True:
            label = "positif"
        else:
            label = "negatif"
#         print(label)
        f.write(str(label+'\n'))